# Read and clean a single file
-------
Nombre: **Roger Galindo**

In [1]:
import os
import sys
import logging
import pandas as pd

from pandas_profiling import ProfileReport


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulator import get_input_data

bucket = 'esp-big-data'
directory = 'BigData'

## Read Metadata
-----
General info for each relevant column

In [115]:
file = 'gs://{}/{}/data/metadatos-llamadas-urg-y-emer.csv'.format(bucket, directory)
df_metadata = pd.read_csv(file, encoding='latin1', sep=';')
df_metadata

,NOMBRE,DESCRIPCION
0,FECHA_INCIDENTE,Es la fecha el cual se registra la llamada del...
1,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,Es la fecha el cual se inicia el desplazamient...
2,CODIGO LOCALIDAD,Es el código de las 20 localidades de la ciuda...
3,LOCALIDAD,Es la localidad donde sucede el incidente.
4,EDAD,La edad del paciente.
5,UNIDAD,"La descripción de la edad si es en horas, días..."
6,GENERO,Es la distinción de genero del paciente
7,RED,Es la localización a nivel bogota de la red de...
8,TIPO_INCIDENTE,Es la descripción inicial que tipifica el cent...
9,PRIORIDAD,Es la tipificación según la prioridad del inci...


The final data must contains this and only this columns with that specific headers in order to standarize the data, this is a type of data dictionary but it also must include the data types (**schema**) of each field (**column**)

## Get raw data

In [233]:
raw_data = get_input_data(
    bucket            = bucket,
    initial_directory = directory,
    filename          = 'datos-abiertos-agosto-2019.csv'
)
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18141 entries, 0 to 18140
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   18141 non-null  int64 
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  18141 non-null  object
 2   CODIGO DE LOCALIDAD                18141 non-null  int64 
 3   LOCALIDAD                          18141 non-null  object
 4   EDAD                               18141 non-null  object
 5   UNIDAD                             18141 non-null  object
 6   GENERO                             18141 non-null  object
 7   RED                                18141 non-null  object
 8   TIPO DE INCIDIDENT                 18141 non-null  object
 9   PRIORIDAD                          18141 non-null  object
 10  MES                                18141 non-null  object
 11  CLASIFICACION FINAL                18138 non-null  object
dtypes: i

It seems that this file contains already the fields, but need to rename them.

September file only contains 10 columns instead of 12, let's impute them

In [234]:
# raw_data['MES'] = ['JULIO']*raw_data.shape[0]                    # fill the column with the same value
# raw_data['CLASIFICACION_FINAL'] = ['SIN-DATO']*raw_data.shape[0]

### Rename Columns

In [235]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL',
        'TIPO DE INCIDIDENT'                : 'TIPO_INCIDENTE',                    # AGOSTO HAS THIS BAD COLUMN NAME
        'FECHA_DESPACHO_518'                : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL', # Octubre and Diciembre has this bad column name
        
    },
    inplace=True
)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


### Check values per field
--------
1. LOCALIDAD
2. GENERO

In [236]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

Kennedy                2717
 Engativa              1805
 Suba                  1678
 Bosa                  1264
Ciudad Bolivar         1220
Puente Aranda           977
 San Cristobal          906
Usaquen                 894
 Rafael Uribe Uribe     892
Fontibon                878
Chapinero               707
Santa Fe                682
 Usme                   637
 Barrios Unidos         608
 Teusaquillo            599
Tunjuelito              595
Los Martires            593
 Antonio Nariño         369
 La Candelaria          119
 Sumapaz                  1
Name: LOCALIDAD, dtype: int64

It seems that in some fields exist a white space at the begining of the values, probably at the end too, lets clean them all

In [237]:
palabra = '   hola mundo    '
palabra.strip()

'hola mundo'

In [238]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2717
Engativa              1805
Suba                  1678
Bosa                  1264
Ciudad Bolivar        1220
Puente Aranda          977
San Cristobal          906
Usaquen                894
Rafael Uribe Uribe     892
Fontibon               878
Chapinero              707
Santa Fe               682
Usme                   637
Barrios Unidos         608
Teusaquillo            599
Tunjuelito             595
Los Martires           593
Antonio Nariño         369
La Candelaria          119
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [239]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [240]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


In [241]:
col = 'GENERO'
raw_data[col].value_counts(dropna=False)

SIN_DATO     8617
MASCULINO    5143
FEMENINO     4381
Name: GENERO, dtype: int64

In [242]:
col = 'GENERO'
raw_data[col].fillna('SIN-DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

SIN_DATO     8617
MASCULINO    5143
FEMENINO     4381
Name: GENERO, dtype: int64

In [243]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    6827
Inconsciente/Paro Cardiorrespiratorio        1875
Dificultad Respiratoria                      1504
Enfermo                                      1233
Convulsiones                                 1135
Heridos                                       847
Lesiones personales                           718
Trastorno mental                              670
Dolor torácico                                628
Intento de suicidio                           555
Síntomas gastrointestinales                   418
Accidente cerebro vascular                    401
Patología Gineco - obstétrica                 353
Caída                                         295
Intoxicaciones                                181
Ideas de suicidio                             102
SIN_DATO                                       58
Incendio estructural                           51
Muerte Natural                                 35
Violencia Sexual                               33


In [244]:
col = 'CLASIFICACION_FINAL'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Traslado                       7311
Cancelado                      4935
Trasladado por Otro Recurso    1813
Falsa Alarma                   1738
Desistimiento                  1449
No Amerita Traslado             452
No Ubica                        235
Fallecido                       205
NaN                               3
Name: CLASIFICACION_FINAL, dtype: int64

Here this columns contains a null value, aka NaN (this is numeric value that is Null), lets replace them first
* NaN --> SIN_DATO

In [245]:
raw_data[col].fillna('SIN-DATO').value_counts()

Traslado                       7311
Cancelado                      4935
Trasladado por Otro Recurso    1813
Falsa Alarma                   1738
Desistimiento                  1449
No Amerita Traslado             452
No Ubica                        235
Fallecido                       205
SIN-DATO                          3
Name: CLASIFICACION_FINAL, dtype: int64

In [246]:
col = 'CLASIFICACION_FINAL'
raw_data[col].fillna('SIN-DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


In [247]:
raw_data['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'] = pd.to_datetime(raw_data['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'], dayfirst=True)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,2019-08-01 00:28:00,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,2019-08-01 00:28:00,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,2019-08-01 00:15:00,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


In [249]:
raw_data['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'].max()

Timestamp('2019-09-01 00:19:00')

# Profile of the Data
------
first install pandas profiling, from a terminal

````
pip install pandas-profiling
````

In [250]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_agosto_2019.html')

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Form the initial report of the file we see that are duplicates rows in the table, we need to subtract them

### Remove duplicates

In [251]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 18141
Number of rows after cleaning: 16807


### Save the final table
use to_csv to save the table

In [252]:
# saving in a local directory
raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_agosto_2019.csv', encoding='latin1', sep=';', index=False)

# Next steps.... create the master table and analyze it